In [1]:
import os
from llama_index.llms.litellm import LiteLLM
from llama_index.core.llms import ChatMessage
from litellm import completion
from llama_index.llms.ollama import Ollama

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import StorageContext, load_index_from_storage

from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

from rich.markdown import Markdown
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
DO_OLLAMA = True
if DO_OLLAMA:
    llm = Ollama(model="llama3.2", request_timeout=60.0)
else:
    llm = LiteLLM("gemini/gemini-2.0-flash")

resp = llm.complete("Who is Paul Graham?")
print (resp)

Paul Graham is an American entrepreneur, investor, and philanthropist. He is best known for co-founding Y Combinator, a prestigious startup accelerator program that has helped launch numerous successful companies.

Graham was born in 1969 in Cambridge, England, to a family of engineers and scientists. He moved to the United States with his family at a young age and grew up in Concord, Massachusetts.

In 1995, Graham co-founded UserLand Software, which developed software applications for web authors, including the popular text editor Scripting News (later known as Scripting Blog). The company was later acquired by Yahoo!.

After UserLand's acquisition, Graham moved on to start another company, Foghorn Systems, but it ultimately failed. He then joined a number of other startups and incubators before launching Y Combinator in 2005.

Y Combinator is a highly selective startup accelerator program that provides funding, mentorship, and resources to early-stage companies. The program has help

In [3]:
documents = SimpleDirectoryReader("./data").load_data()


In [4]:
if DO_OLLAMA:
    Settings.embed_model = OllamaEmbedding(model_name="llama3.2")
else:
    Settings.embed_model = GeminiEmbedding(model_name="models/embedding-001")

# Create and save to chroma
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

vector_index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    show_progress=True
)

query_engine = vector_index.as_query_engine(llm=llm)

Parsing nodes:   0%|          | 0/26 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/55 [00:00<?, ?it/s]

InvalidDimensionException: Embedding dimension 3072 does not match collection dimensionality 768

In [ ]:
response = query_engine.query("What is the revolver concept, and why is it robust? What are it's main features? Respond in markdown.")
Markdown(response.response)

I am sorry, but this document does not contain the answer to your question about the revolver concept.

In [ ]:
query_engine?

Type:        RetrieverQueryEngine
String form: <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x35fc6f620>
File:        ~/Documents/Github/100-days-of-grind/.venv/lib/python3.12/site-packages/llama_index/core/query_engine/retriever_query_engine.py
Docstring:  
Retriever query engine.

Args:
    retriever (BaseRetriever): A retriever object.
    response_synthesizer (Optional[BaseSynthesizer]): A BaseSynthesizer
        object.
    callback_manager (Optional[CallbackManager]): A callback manager.

In [ ]:
response = query_engine.query("What is the convergence ratio for a double shell capsule?")
Markdown(response.response)


The total convergence ratio for a double shell is larger than that of a single shell, with CRTot /C24 55 for a     
fall-line optimized double shell versus CRTot /C24 40 for a typical single shell.

In [5]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)



In [6]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True,return_intermediate_steps=True)

In [7]:
response = agent.chat("What is 20+(2*4)? Use a tool to calculate every step.")

> Running step 3a3eb639-9286-49ed-b370-86ce730e1934. Step input: What is 20+(2*4)? Use a tool to calculate every step.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step f9c6ed9d-ab9b-4739-9f33-23804844eda0. Step input: None
Thought: The current result from the multiplication tool is 8. Now I can continue with the original expression.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 040eedf5-819c-45a7-bec9-e7fc831961fd. Step input: None
Thought: The current result from the addition tool is 28. Now I have the final answer to the original question.
Answer: 28


In [8]:
from llama_index.tools.code_interpreter import CodeInterpreterToolSpec

code_spec = CodeInterpreterToolSpec()

agent = ReActAgent.from_tools(code_spec.to_tool_list(), llm=llm, verbose=True,return_intermediate_steps=True)

In [9]:
response = agent.chat(
    "write a python function to calculate volume of a sphere with radius 4.3cm"
)

> Running step b71944b9-ffde-4b0b-9d12-96db71ab11f2. Step input: write a python function to calculate volume of a sphere with radius 4.3cm
Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: code_interpreter
Action Input: {'code': 'import math\ndef sphere_volume(radius):\n    return (4/3) * math.pi * radius**3\ncalculate_volume = sphere_volume(4.3)\npres = round(calculate_volume, 2)'}
Observation: StdOut:
b''
StdErr:
b''
> Running step 45d9abc3-0f2e-4d17-9949-ea4a101130a9. Step input: None
Thought: The tool execution was successful but did not produce any output. This could be due to the lack of standard output in this specific case.
Action: code_interpreter
Action Input: {'code': 'import math\ndef sphere_volume(radius):\n    return (4/3) * math.pi * radius**3\ncalculate_volume = sphere_volume(4.3)\npres = round(calculate_volume, 2)\nprint(pres)'}
Observation: StdOut:
b'333.04\n'
StdErr:
b''
> Running step 8f0a15a5-b2bf-40

In [10]:
Markdown(response.response)

The volume of a sphere with radius 4.3cm is approximately 333.04 cubic centimeters.

In [ ]:
response = agent.chat(
    "Write a python function that scrapes the html code from https://www.google.com, and prints it."
)

> Running step 0b9f02bb-c680-43c3-a419-47e2f67db266. Step input: Write a python function that scrapes the html code from https://www.google.com, and prints it.
Thought: The user is asking me to write a Python function to scrape HTML code from a website. I can use the `requests` library to fetch the HTML content and then print it. I should also handle potential errors like connection issues.
Action: code_interpreter
Action Input: {'code': 'import requests\n\ndef scrape_website(url):\n  """Scrapes the HTML code from a given URL and prints it.\n\n  Args:\n    url: The URL of the website to scrape.\n\n  Returns:\n    None. Prints the HTML content to the console.\n  """\n  try:\n    response = requests.get(url)\n    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)\n    html_content = response.text\n    print(html_content)\n  except requests.exceptions.RequestException as e:\n    print(f"Error scraping {url}: {e}")\n\n# Example usage:\nurl = "https://www.google.c

In [ ]:
Markdown(response.response)

                                                                                                                   
 import requests                                                                                                   
                                                                                                                   
 def scrape_website(url):                                                                                          
   """Scrapes the HTML code from a given URL and prints it.                                                        
                                                                                                                   
   Args:                                                                                                           
     url: The URL of the website to scrape.                                                                        
                                                                                                                   
   Returns:                                                                                                        
     None. Prints the HTML content to the console.                                                                 
   """                                                                                                             
   try:                                                                                                            
     response = requests.get(url)                                                                                  
     response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)                                 
     html_content = response.text                                                                                  
     print(html_content)                                                                                           
   except requests.exceptions.RequestException as e:                                                               
     print(f"Error scraping {url}: {e}")                                                                           
                                                                                                                   
 # Example usage:                                                                                                  
 url = "https://www.google.com"                                                                                    
 scrape_website(url)                                                                                               
                                                                                                                   

This function uses the requests library to fetch the HTML content from the specified URL. It includes error        
handling to catch potential issues during the request. The scraped HTML content is then printed to the console.

In [ ]:
import requests

def scrape_website(url):
  """Scrapes the HTML code from a given URL and prints it.

  Args:
    url: The URL of the website to scrape.

  Returns:
    None. Prints the HTML content to the console.
  """
  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    html_content = response.text                                                                                  
    print(html_content) 
  except requests.exceptions.RequestException as e:                                                               
     print(f"Error scraping {url}: {e}")                                                                           
                                                                                                                   
# Example usage:                                                                                                  
url = "https://www.google.com"                                                                                    
scrape_website(url)     

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for." name="description"><meta content="noodp, " name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="oc1mQUlKC-UfZqP3kvPmhA">(function(){var _g={kEI:'M9e0Z9dj3MuQ8g-H14fABQ',kEXPI:'0,793110,2907173,1101,538661,2872,2891,43028,30022,16105,78219,266577,212051,47082,30911,5230292,11229,166,16272881,20539756,25228681,11731,53725,57648,884,14280,8181,5934,4559,7383,10970,20584,19011,2656,3438,3319,23879,7033,2105,3078,1522,328,6225,11435,22876,29854,1341,5328,8380,8210,3286,4134,21306,33,9041,17667,10666,21345,4486,3854,41,13639,1,4362,2379,4106,7736,11565,2,5799,950,2147,4610,7,577

In [11]:
from llama_index.core.agent.workflow import (
    AgentWorkflow,
    FunctionAgent,
    ReActAgent,
)
from llama_index.core.tools import FunctionTool


# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b


# Create agent configs
# NOTE: we can use FunctionAgent or ReActAgent here.
# FunctionAgent works for LLMs with a function calling API.
# ReActAgent works for any LLM.
calculator_agent = FunctionAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant.",
    tools=[
        FunctionTool.from_defaults(fn=add),
        FunctionTool.from_defaults(fn=subtract),
    ],
    llm=llm,
)

retriever_agent = FunctionAgent(
    name="retriever",
    description="Manages data retrieval",
    system_prompt="You are a retrieval assistant.",
    llm=llm,
)

# Create and run the workflow
workflow = AgentWorkflow(
    agents=[calculator_agent, retriever_agent], root_agent="calculator"
)

# Run the system
response = await workflow.run(user_msg="Can you add 5 and 3?")

#  Or stream the events
handler = workflow.run(user_msg="Can you add 5 and 3?")
async for event in handler.stream_events():
    if hasattr(event, "delta"):
        print(event.delta, end="", flush=True)

The result of adding 5 and 3 is 8.

In [13]:
response = workflow.run(user_msg="Can you add 5 and 3?")
response

<WorkflowHandler pending cb=[Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-9e195855811e', bound_args=<BoundArgumen...mory': None})>, instance=<llama_index....t 0x1629970b0>, context=<_contextvars...t 0x1633b4f80>)() at /Users/jsmidt/Documents/Github/100-days-of-grind/.venv/lib/python3.12/site-packages/llama_index/core/instrumentation/dispatcher.py:274]>

In [16]:
response

<WorkflowHandler finished result=AgentOutput(r...='calculator')>